In [1]:
import numpy as np
from numba import jit
import sympy

# Item XV

Considering the following inner product:
$$
\langle p(x),q(x) \rangle =\int_{-1}^{1} \overline{p(x)}q(x) dx
$$

* Let $A= [1|x|x^2|...|x^{n-1}]$ be the "matrix" whose "columns" are the monomials $x^j$, for $j=0,...,n-1$. Each column is a function in $L^2[-1,1]$. compute the $QR$ decomposition of $A$.
* Let $A=[1|\sin(2\pi x)|\sin(4\pi x)|...|x^{n-1}]$ be the "matrix" whose "columns" are the functions $1$ and $\sin(2\pi x)$, for $j=1,...,n-1$. Each column is a function in $L^2[-1,1]$. Compute the $QR$ decomposition of $A$. 
* Do part (a) numerically. Make sure you understand what you are doing since this is a important concept that links symbolic computing with numerical computing.

---

In [2]:
# This is a generic version of Gram-Schmidt, by default it works on matrices.
# For other uses, replace default argument functions.
def generic_gs(
        elems,
        scalar = lambda a,x : a*x,
        prod = lambda x,y : np.sum(x*y),
        neg = lambda x,y : x-y,
    ):
    """
    elems = [T]
    scalar :: T -> Float -> T
    prod :: T -> T -> Float
    neg :: T -> T -> T
    NOTE: if is used for a regular matrix, elems must be row-wise.
    """
    n = len(elems)
    r = np.zeros((n,n))
    for i in range(n):
        for j in range(i):
            projection = prod(elems[j],elems[i])/prod(elems[j],elems[j])
            r[j,i] = projection
            elems[i] = neg(elems[i],scalar(projection,elems[j]))
        norm2 = prod(elems[i],elems[i])
        if norm2<0:
            print("Warning: negative norm2=%f at i=%d!"%(norm2,i))
            return None
        norm = norm2**0.5
        r[i,i] = norm
        elems[i] = scalar(norm**-1,elems[i])
    return r

In [3]:
def symbolic_inner_product(f,q):
    x = sympy.Symbol('x')
    v = sympy.integrate(f*q,(x,-1,1))
    # We evaluate the expresion as a number because we can't afford the whole symbolic
    # expression...
    return float(v)

In [4]:
# We define the list of functions for part a
def part_a_funcs(n):
    x = sympy.Symbol('x')
    part = [x**i for i in range(n)]
    return part

In [5]:
def part_b_funcs(n):
    x = sympy.Symbol('x')
    part = [x**0] + [sympy.sin(2*i*sympy.pi*x) for i in range(1,n)]
    return part

In [6]:
# We can print an array of functions:
print(part_a_funcs(10))
print(part_b_funcs(10))

[1, x, x**2, x**3, x**4, x**5, x**6, x**7, x**8, x**9]
[1, sin(2*pi*x), sin(4*pi*x), sin(6*pi*x), sin(8*pi*x), sin(10*pi*x), sin(12*pi*x), sin(14*pi*x), sin(16*pi*x), sin(18*pi*x)]


In [7]:
# We compute the decompositions
FUNCS = [part_a_funcs(5),part_b_funcs(5),
         part_a_funcs(20),part_b_funcs(20),
         part_a_funcs(100),part_b_funcs(100)]
for funcs in FUNCS:
    # Print Original matrix
    print("-"*20+"Functions:")
    print(funcs)
    # Perform QR decomposition using generic G-S
    r = generic_gs(funcs,prod=symbolic_inner_product)
    if r is None:
        print("Couldn't compute R!!!:")
    else:
        # Print Q
        print("Q:")
        print(funcs)
        # Print R
        print("R:")
        print(r)

--------------------Functions:
[1, x, x**2, x**3, x**4]
Q:
[0.707106781186547, 1.22474487139159*x, 2.37170824512628*x**2 - 0.790569415042095, 4.67707173346743*x**3 - 2.80624304008046*x, 9.28077650307342*x**4 - 7.95495128834865*x**2 + 0.795495128834865]
R:
[[1.41421356 0.         0.47140452 0.         0.28284271]
 [0.         0.81649658 0.         0.48989795 0.        ]
 [0.         0.         0.42163702 0.         0.36140316]
 [0.         0.         0.         0.21380899 0.        ]
 [0.         0.         0.         0.         0.1077496 ]]
--------------------Functions:
[1, sin(2*pi*x), sin(4*pi*x), sin(6*pi*x), sin(8*pi*x)]
Q:
[0.707106781186547, 1.0*sin(2*pi*x), 1.0*sin(4*pi*x), 1.0*sin(6*pi*x), 1.0*sin(8*pi*x)]
R:
[[1.41421356 0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.        ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.   

Q:
[0.707106781186547, 1.0*sin(2*pi*x), 1.0*sin(4*pi*x), 1.0*sin(6*pi*x), 1.0*sin(8*pi*x), 1.0*sin(10*pi*x), 1.0*sin(12*pi*x), 1.0*sin(14*pi*x), 1.0*sin(16*pi*x), 1.0*sin(18*pi*x), 1.0*sin(20*pi*x), 1.0*sin(22*pi*x), 1.0*sin(24*pi*x), 1.0*sin(26*pi*x), 1.0*sin(28*pi*x), 1.0*sin(30*pi*x), 1.0*sin(32*pi*x), 1.0*sin(34*pi*x), 1.0*sin(36*pi*x), 1.0*sin(38*pi*x)]
R:
[[1.41421356 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.      

We can see that the functions of the second item are ortogonal, so the $QR$ decomposition gives the indentity (besides the first function $y(x)=1$ that has to be normalized).

Around $i=25$ the function coeficients become too small to handle. The norm (inner product with itself) of the functions after substracting the projections becomes small, and negative.

In [8]:
FUNCS = [part_a_funcs(100),part_b_funcs(100)]
for funcs in FUNCS:
    # Print Original matrix
    print("-"*20+"Functions:")
    print(funcs)
    # Perform QR decomposition using generic G-S
    r = generic_gs(funcs,prod=symbolic_inner_product)
    if r is None:
        print("Couldn't compute R!!!:")
    else:
        # Print Q
        print("Q:")
        print(funcs)
        # Print R
        print("R:")
        print(r)

--------------------Functions:
[1, x, x**2, x**3, x**4, x**5, x**6, x**7, x**8, x**9, x**10, x**11, x**12, x**13, x**14, x**15, x**16, x**17, x**18, x**19, x**20, x**21, x**22, x**23, x**24, x**25, x**26, x**27, x**28, x**29, x**30, x**31, x**32, x**33, x**34, x**35, x**36, x**37, x**38, x**39, x**40, x**41, x**42, x**43, x**44, x**45, x**46, x**47, x**48, x**49, x**50, x**51, x**52, x**53, x**54, x**55, x**56, x**57, x**58, x**59, x**60, x**61, x**62, x**63, x**64, x**65, x**66, x**67, x**68, x**69, x**70, x**71, x**72, x**73, x**74, x**75, x**76, x**77, x**78, x**79, x**80, x**81, x**82, x**83, x**84, x**85, x**86, x**87, x**88, x**89, x**90, x**91, x**92, x**93, x**94, x**95, x**96, x**97, x**98, x**99]
Couldn't compute R!!!:
--------------------Functions:
[1, sin(2*pi*x), sin(4*pi*x), sin(6*pi*x), sin(8*pi*x), sin(10*pi*x), sin(12*pi*x), sin(14*pi*x), sin(16*pi*x), sin(18*pi*x), sin(20*pi*x), sin(22*pi*x), sin(24*pi*x), sin(26*pi*x), sin(28*pi*x), sin(30*pi*x), sin(32*pi*x), sin(34

---
To do it numerically, let's define a polynomial
$$
p(x) = \sum_{i=0}^{n-1} p_i x^i
$$
as the array of the $p_i$'s.

Then the multiplication becomes:
$$
p(x)q(x) =  \sum_{i=0}^{2n-2} \left( \sum_{k=0}^{i} p_k q_{i-k} \right) x^{i} \,,
$$
then the inner product becomes:
\begin{align*}
\int_{-1}^{1} p(x)q(x) \, dx &= \sum_{i=0}^{2n-2} \left( \sum_{k=0}^{i} p_k q_{i-k} \right) \frac{x^{i+1}}{i+1} |_{x=-1}^{1}
\\ &= \sum_{i=0}^{2n-2} [i \, \text{mod} \, 2= 0] \left( \sum_{k=0}^{i} p_k q_{i-k} \right) \frac{2}{i+1}
\end{align*}

In [9]:
@jit(nopython=True)
def poly_mult(a,b):
    assert(len(a)==len(b))
    n = len(a)
    total = 0
    for i in range(0,2*n-1,2):
        term = 0
        for k in range(0,i+1):
            if k>=0 and i-k>=0 and k<n and i-k<n:
                term += a[k]*b[i-k]
        term *= 2.0/(i+1.0)
        total += term
    return total

In [10]:
def poly_print(poly):
    stri = []
    if len(poly)>0 and poly[0] != 0: stri.append("%.3f"%poly[0])
    if len(poly)>1 and poly[1] != 0: stri.append("%.3fx"%poly[1])
    for i in range(2,len(poly)):
        if poly[i]!=0:
            stri.append("%+.3fx%d"%(poly[i],i))
    if len(stri)==0: return "0"
    return " ".join(stri)

def poly_matrix_print(polys,limit=10):
    print("[")
    if len(polys)>2*limit:
        for poly in polys[:limit]:
            print("  "+poly_print(poly)+" |")
        print("  ...")
        for poly in polys[-limit:]:
            print("  "+poly_print(poly)+" |")
    else:
        for poly in polys:
            print("  "+poly_print(poly)+" |")
    print("]")

In [11]:
poly_mult([1,2,3],[2,5,1])

16.53333333333333

In [12]:
for N in (5,10,100):
    # Print Original matrix
    polys = np.eye(N)
    print("-"*20+" Original (N=%d):"%N)
    poly_matrix_print(polys)
    # Perform QR decomposition using generic G-S
    r = generic_gs(polys,prod=poly_mult)
    # Print Q
    print("Q:")
    poly_matrix_print(polys)
    # Assert that Q is orthonormal
    for i in range(N):
        for j in range(N):
            if i==j:
                assert(np.abs(poly_mult(polys[i],polys[j])-1)<1e-5)
            else:
                assert(np.abs(poly_mult(polys[i],polys[j]))<1e-5)
    # Print R
    print("R:")
    print(r)

-------------------- Original (N=5):
[
  1.000 |
  1.000x |
  +1.000x2 |
  +1.000x3 |
  +1.000x4 |
]
Q:
[
  0.707 |
  1.225x |
  -0.791 +2.372x2 |
  -2.806x +4.677x3 |
  0.795 -7.955x2 +9.281x4 |
]
R:
[[1.41421356 0.         0.47140452 0.         0.28284271]
 [0.         0.81649658 0.         0.48989795 0.        ]
 [0.         0.         0.42163702 0.         0.36140316]
 [0.         0.         0.         0.21380899 0.        ]
 [0.         0.         0.         0.         0.1077496 ]]
-------------------- Original (N=10):
[
  1.000 |
  1.000x |
  +1.000x2 |
  +1.000x3 |
  +1.000x4 |
  +1.000x5 |
  +1.000x6 |
  +1.000x7 |
  +1.000x8 |
  +1.000x9 |
]
Q:
[
  0.707 |
  1.225x |
  -0.791 +2.372x2 |
  -2.806x +4.677x3 |
  0.795 -7.955x2 +9.281x4 |
  4.397x -20.521x3 +18.469x5 |
  -0.797 +16.731x2 -50.193x4 +36.809x6 |
  -5.991x +53.916x3 -118.616x5 +73.429x7 |
  0.797 -28.699x2 +157.846x4 -273.599x6 +146.571x8 |
  7.585x -111.248x3 +433.869x5 -619.813x7 +292.689x9 |
]
R:
[[1.41421356 0.   

AssertionError: 

We can see that the numerical method fails for $N=100$. After a closer inspection, this was because, after substracting the projection with the previous functions, the remaining polynomial had very small coefficients, around $i=25$ too.